# 305 Prepare equity metric data

In this notebook, we pull out the R10 data for the following variables:
* Emissions|CO2|Energy and Industrial Processes
* Population
We do this across the set of C1 scenarios assessed in this study (100_scenario_names.csv)

In [1]:
import pyam
import pandas as pd
import numpy as np

from pathlib import Path
import os
import dotenv

<IPython.core.display.Javascript object>

In [2]:
dotenv.load_dotenv()

True

Step 1: Read in the AR6 R10 data and downselect to the necessary variables.

In [3]:
ar6_r10 = pd.read_csv(
    os.environ['AR6_R10']
)

In [4]:
ar6_r10_vars = (
    ar6_r10[ar6_r10['Variable'].isin(
        ['Emissions|CO2|Energy and Industrial Processes', 'Population']
    )]
)

In [5]:
ar6_r10_pyam = pyam.IamDataFrame(ar6_r10_vars)

In [6]:
ar6_r10_pyam.load_meta(
    os.environ['AR6_META']
)

pyam.core - WARNING: No meta indicators for the following scenarios:
          model             scenario
0  GCAM-USA 5.3  EN_NP_2025_-1002050
1  GCAM-USA 5.3   EN_NP_2025_-502050
2  GCAM-USA 5.3             EN_NP_BL
3  GCAM-USA 5.3         EN_NP_CurPol
4  GCAM-USA 5.3           EN_NP_UNDC
...
pyam.core - WARNING: Ignoring meta indicators for the following scenarios:
         model                scenario
0  AIM/CGE 2.0  ADVANCE_2020_1.5C-2100
1  AIM/CGE 2.0      ADVANCE_2020_Med2C
2  AIM/CGE 2.0       ADVANCE_2020_WB2C
3  AIM/CGE 2.0      ADVANCE_2030_Med2C
4  AIM/CGE 2.0  ADVANCE_2030_Price1.5C
...


Step 2: Filter for the C1 scenarios.

In [7]:
ar6_c1 = ar6_r10_pyam.filter(
    Category='C1'
)

Step 3: Check which scenarios report all the R10 regions.

In [8]:
regions = set(ar6_c1.region) - set(['R10ROWO'])

In [9]:
ar6_c1.require_data(
    region=regions,
    exclude_on_fail=True
)

pyam.validation - INFO: 2 scenarios failed validation and will be set as `exclude=True`.


,model,scenario,region
0,GEM-E3_V2021,EN_NPi2020_500,R10AFRICA
1,GEM-E3_V2021,EN_NPi2020_600_COV,R10AFRICA


In [10]:
ar6_c1.filter(
    exclude=False,
    inplace=True
)

In [11]:
ar6_c1.interpolate(
    time=range(2015, 2101, 5),
    inplace=True
)

Step 4: Write this out for further assessment.

In [12]:
ar6_c1.to_csv(
    Path(
        '../data/305_equity_input.csv'
    )
)